# Jonathan Halverson
# Friday, September 9, 2016
# Spark 2

In [2]:
lines = sc.textFile('text_file.md')
print lines.count()
print lines.first()

95
# Apache Spark


In [6]:
print lines.take(5)

[u'# Apache Spark', u'', u'Spark is a fast and general cluster computing system for Big Data. It provides', u'high-level APIs in Scala, Java, Python, and R, and an optimized engine that', u'supports general computation graphs for data analysis. It also supports a']


In [7]:
print lines.collect()

[u'# Apache Spark', u'', u'Spark is a fast and general cluster computing system for Big Data. It provides', u'high-level APIs in Scala, Java, Python, and R, and an optimized engine that', u'supports general computation graphs for data analysis. It also supports a', u'rich set of higher-level tools including Spark SQL for SQL and DataFrames,', u'MLlib for machine learning, GraphX for graph processing,', u'and Spark Streaming for stream processing.', u'', u'<http://spark.apache.org/>', u'', u'', u'## Online Documentation', u'', u'You can find the latest Spark documentation, including a programming', u'guide, on the [project web page](http://spark.apache.org/documentation.html)', u'and [project wiki](https://cwiki.apache.org/confluence/display/SPARK).', u'This README file only contains basic setup instructions.', u'', u'## Building Spark', u'', u'Spark is built using [Apache Maven](http://maven.apache.org/).', u'To build Spark and its example programs, run:', u'', u'    build/mvn -DskipTe

In [8]:
lines.sample(withReplacement=False, 0.5)

SyntaxError: non-keyword arg after keyword arg (<ipython-input-8-3e27512fb14b>, line 1)